In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import dill
from tqdm import tqdm
import sys
import os
import gc
import psutil  # Add this import to monitor memory usage

# Get the current working directory
current_dir = os.getcwd()
# Add the parent directory to sys.path
sys.path.append(os.path.dirname(current_dir))

In [2]:
from analyze.report import draw_rate_vs_angle_pop

/home/utente/sl_plots/hears_env_linux/lib/python3.12/site-packages/setuptools/_distutils/_msvccompiler.py:12: UserWarning: _get_vc_env is private; find an alternative (pypa/distutils#340)
  warnings.warn(


In [3]:
# List directory contents
result_dir = '/mnt/c/Users/utente/OneDrive - Politecnico di Milano/work/sound_localization/sl_results/'
items = os.listdir(result_dir)
for item in items:
    print(item) 


clicks_1000_XXdB&TanCarney&subject_1.pic
harmonic_69dB&TanCarney&subject_1.pic
tone_0.1kHz_70dB&TanCarney&ild_only.pic
tone_0.1kHz_70dB&TanCarney&itd_only.pic
tone_0.1kHz_70dB&TanCarney&itd_only_myoga_inv.pic
tone_0.1kHz_70dB&TanCarney&itd_only_myoga_null.pic
tone_0.1kHz_70dB&TanCarney&itd_only_no_MSO_inh.pic
tone_0.1kHz_70dB&TanCarney&subject_1.pic
tone_0.1kHz_70dB&TanCarney&subject_1_myoga_inv.pic
tone_0.1kHz_70dB&TanCarney&subject_1_myoga_null.pic
tone_0.1kHz_70dB&TanCarney&subject_1_no_MSO_inh.pic
tone_0.1kHz_70dB&TanCarney&subject_2.pic
tone_0.1kHz_70dB&TanCarney&subject_3.pic
tone_0.1kHz_70dB&TanCarney&subject_4.pic
tone_0.1kHz_70dB&TanCarney&subject_5.pic
tone_0.2kHz_70dB&TanCarney&subject_1.pic
tone_0.3kHz_70dB&TanCarney&subject_1.pic
tone_0.4kHz_70dB&TanCarney&subject_1.pic
tone_0.5kHz_70dB&TanCarney&subject_1.pic
tone_0.6kHz_70dB&TanCarney&subject_1.pic
tone_0.7kHz_70dB&TanCarney&subject_1.pic
tone_0.8kHz_70dB&TanCarney&subject_1.pic
tone_0.9kHz_70dB&TanCarney&subject_1.pic
t

In [4]:
sounds = ['tone_0.1kHz', 'tone_1.kHz', 'whitenoise']
cf_intervals = [[75,125], [900,1100], None] #[20,20000]

In [5]:
def get_memory_usage():
    """Return current memory usage in a human-readable format"""
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    return f"Memory usage: {memory_info.rss / (1024 * 1024):.2f} MB"

In [8]:
for f, cf_interval in zip(sounds, cf_intervals):
    try:
        print(f"Starting processing for {f}")
        print(get_memory_usage())
        
        # Create a single figure and axis
        fig, ax = plt.subplots(figsize=(7, 4))
        print(f, '\n')

        file_infos = [
            {
                "path": result_dir + f"{f}_70dB&TanCarney&itd_only.pic",
                "label": "Control",
                "color": "m"
            },
            {
                "path": result_dir + f"{f}_70dB&TanCarney&itd_only_myoga_null.pic",
                "label": "Null Delays",
                "color": "y"
            },
            {
                "path": result_dir + f"{f}_70dB&TanCarney&itd_only_myoga_inv.pic",
                "label": "Inverted Delays",
                "color": "b"
            }
        ]

        # Track all original values (rates) for reporting
        all_original_values = {}
        
        # Process files one at a time
        for info in file_infos:
            try:
                print(f"Processing {info['path'].split('/')[-1]}...")
                print(get_memory_usage())
                
                # Check if file exists
                if not os.path.exists(info["path"]):
                    print(f"Warning: File {info['path']} does not exist. Skipping.")
                    continue
                
                # Load single file with better error handling
                try:
                    with open(info["path"], "rb") as file:
                        result = dill.load(file, ignore=True)
                except Exception as e:
                    print(f"Error loading file {info['path']}: {e}")
                    continue
        
                # Pass the axis to the function and specify the color and custom label
                ax, original_values = draw_rate_vs_angle_pop(
                    data=result,
                    color=info["color"],
                    pop='MSO',
                    cf_interval=cf_interval,
                    rate=True,
                    sides='L',
                    norm=True,
                    ax=ax,
                    title="",  # No title for individual plots
                    label=info["label"]  # Use the custom label
                )
                
                # Store original values for reporting
                if original_values:
                    all_original_values[info["label"]] = original_values
                    
                    # Print the min/max rate values
                    for side in original_values:
                        min_value = original_values[side]['min_value']
                        max_value = original_values[side]['max_value']
                        print(f"{info['label']} - Side {side}:")
                        print(f"  Min rate: {min_value:.2f} Hz")
                        print(f"  Max rate: {max_value:.2f} Hz")
                
                # Explicit memory cleanup
                del result
                gc.collect()
                print(f"Processed and plotted {info['label']}")
                print(get_memory_usage())
                
            except Exception as e:
                print(f"Error processing {info['label']}: {e}")
                continue
    
        # Finalize and save the plot
        ax.legend()
        plt.tight_layout()
        
        # Save the plot
        #plt.savefig(f"blockage_{f}.png", dpi=300)
        plt.show()
        # Explicit cleanup
        plt.close(fig)
        gc.collect()
        print(f"Completed processing for {f}")
        print(get_memory_usage())
        
    except Exception as e:
        print(f"Error during processing of {f}: {e}")
        # Make sure to clean up resources even if there's an error
        plt.close('all')
        gc.collect()
        
    # Additional cleanup between iterations
    plt.close('all')  # Close all figures to be safe
    gc.collect()
    print("--------------------")

Starting processing for tone_0.1kHz
Memory usage: 3740.29 MB
tone_0.1kHz 

Processing tone_0.1kHz_70dB&TanCarney&itd_only.pic...
Memory usage: 3740.53 MB


: 